In [1]:
from utils.utils import build_market_image, prepare_forecast_data
# from environments.live_environments import LiveTradingEnv
from utils.clients import OandaClient
from Keys import *
# from utils.mappings import oanda_time_map
from utils.reward_functions import sharpe_reward_function
import pandas as pd
from json import dumps

import time
import datetime



need to install alpaca


In [2]:
# from neuralforecast.core import NeuralForecast
# forecasting_model=NeuralForecast.load('MultiHeadForecastingModel/')
# agent_dir='Agent/final_checkpoints/policies/default_policy'


In [3]:

from configs import defaults

base_asset=defaults.base_asset
quote_asset=defaults.quote_asset
test_net=True
time_frame=defaults.time_frame
positions=defaults.env_config['positions']

product_type=defaults.product_type

exchange=defaults.exchange
trade_target='/'.join([base_asset,quote_asset]) 
agent_path=defaults.agent_path
coin_pairs=defaults.COIN_PAIRS
trade_target

'USD/JPY'

In [4]:
client=OandaClient(api_key=oanda_api_key,
                    account_id=oanda_account_id,
                    paper=test_net,
                    symbol=trade_target,
                    time_frame=time_frame,
                    product_type=product_type

                    )
client.trade_client,client.base_asset,client.quote_asset,client.symbol,client.time_frame,client.oanda_time_frame



(<oandapyV20.oandapyV20.API at 0x11912a590>,
 'USD',
 'JPY',
 'USD_JPY',
 '1h',
 'H1')

In [5]:
from oandapyV20.endpoints import accounts, orders, pricing, instruments,transactions
import re

In [6]:
account=client.update_account()
account

,balance,pl,unrealizedPL,NAV,marginUsed,marginAvailable,positionValue,dividendAdjustment,financing,id,initialMarginRequired,initialUnits,openTime,price,realizedPL,state
USD,9985.5252,-19.9431,-253.2716,9732.2536,498.9500,9235.1615,9979.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JPY,9979,NaN,-253.2716,NaN,498.9500,NaN,NaN,0.0000,5.7661,129,498.9500,9979,2024-11-25T23:25:04.046400159Z,154.017,0.0000,OPEN


In [7]:
client._position_frame

,averagePrice,dividendAdjustment,financing,guaranteedExecutionFees,pl,resettablePL,side,tradeIDs,balance,unrealizedPL
symbol,,,,,,,,,,
USD_JPY,154.090,0.0000,-0.3041,0.0000,-2.4310,-2.4310,sell,[71],-1464,-0.8589


In [8]:
# req=transactions.TransactionList(accountID=client.account_id)

# last_id=client.trade_client.request(req)['lastTransactionID']
# first_id='1'
# params={'from':first_id,'to':last_id}
# t_req=transactions.TransactionIDRange(accountID=client.account_id,params=params)
# trans_history=client.trade_client.request(t_req)
# trans_history['transactions']
# trans_frame=pd.DataFrame(trans_history['transactions'])
# trans_frame
# trans_frame.to_pickle(f'data_forex/oanda-{client.symbol}-{time_frame}-trans_history.pkl')


In [9]:
client.get_forex_position()

,averagePrice,dividendAdjustment,financing,guaranteedExecutionFees,pl,resettablePL,side,tradeIDs,balance,unrealizedPL
symbol,,,,,,,,,,
USD_JPY,154.090,0.0000,-0.3041,0.0000,-2.4310,-2.4310,sell,[71],-1464,-0.8589


-0.044289367869961155

In [10]:
# for symbol in coin_pairs:
#     print(symbol)
#     data=client.get_historical_data(symbol=symbol,start_date=datetime.datetime(2024,1,1))
#     display(data.head(5))
#     pair_name=re.sub(r'/|-|_','',symbol)
#     data.to_pickle(f'data_forex/oanda-{pair_name}-{time_frame}.pkl')



In [11]:
resp=client.buy(symbol='USD_JPY',quote_size=1)


{"orderRejectTransaction":{"id":"72","accountID":"101-001-22524644-006","userID":22524644,"batchID":"72","requestID":"97338894170720384","time":"2024-11-25T19:52:39.124664670Z","type":"MARKET_ORDER_REJECT","rejectReason":"UNITS_INVALID","instrument":"USD_JPY","units":"0","timeInForce":"FOK","positionFill":"DEFAULT","reason":"CLIENT_ORDER"},"relatedTransactionIDs":["72"],"lastTransactionID":"72","errorMessage":"Order units specified are invalid","errorCode":"UNITS_INVALID"}


In [12]:
resp

{'success': False,
 'error': '{"orderRejectTransaction":{"id":"72","accountID":"101-001-22524644-006","userID":22524644,"batchID":"72","requestID":"97338894170720384","time":"2024-11-25T19:52:39.124664670Z","type":"MARKET_ORDER_REJECT","rejectReason":"UNITS_INVALID","instrument":"USD_JPY","units":"0","timeInForce":"FOK","positionFill":"DEFAULT","reason":"CLIENT_ORDER"},"relatedTransactionIDs":["72"],"lastTransactionID":"72","errorMessage":"Order units specified are invalid","errorCode":"UNITS_INVALID"}'}

In [13]:
params={"instruments":client.symbol}

req=instruments.Instrument(client.account_id, params=params)
trade_info=client.trade_client.request(req)
trade_info.keys()
pricing


AttributeError: module 'oandapyV20.endpoints.instruments' has no attribute 'Instrument'

In [ ]:
strip_symbol=r'/|_|-|USD'
symbol='USD-JPY'
base_asset= re.sub(strip_symbol,'',symbol)
base_asset


In [37]:
def strip_quote(symbol,quote_asset):
    if symbol==quote_asset:
        return symbol
    else:
        strip_pattern=f'/|_|-|{quote_asset}$'
        asset=re.sub(strip_pattern,'',symbol)
        return asset



In [ ]:
[strip_quote(symbol,client.base_asset) for symbol in client._account.index]

In [ ]:

account_frame=pd.DataFrame([account])

account_frame.rename(columns={'currency':'symbol'},inplace=True)
account_frame.set_index('symbol',inplace=True)

account_frame=account_frame[['balance','pl','unrealizedPL','NAV','marginUsed','marginAvailable','positionValue']]

account_frame

In [ ]:
trade_frame=pd.DataFrame(trades)
trade_frame.rename(columns={'instrument':'symbol','currentUnits':'balance'},inplace=True)
trade_frame.set_index('symbol',inplace=True)
trade_frame




In [ ]:
full_account_frame=pd.concat([account_frame,trade_frame],axis=0)

full_account_frame

In [ ]:
positions=client.trade_client.list_futures_positions()
positions=pd.DataFrame(positions['positions'])
positions=positions.set_index('product_id')
positions

In [ ]:
positions

In [ ]:

product_frame=pd.DataFrame(products['products'])
product_frame=product_frame[product_frame['view_only']==False]

# product_frame=product_frame[product_frame['trading_disabled']==False]

product_frame

In [ ]:
trade_info

In [ ]:
# products=client.trade_client.get_products()
now=datetime.datetime.now()
start_time=now-datetime.timedelta(hours=input_size*2)

int(now.timestamp())

In [ ]:
fills=client.get_orders()
fills

In [ ]:
time.sleep(10)

In [ ]:
client.sell('BTC-USDC',base_size='.000014')

In [ ]:
time.sleep(10)

In [ ]:
fills=client.get_orders()
fills